# Manipulation.py: Pushing example

This tutorial provides an example of how to plan a pushing motion on a block using the manipulation.py helper functions

As always we start with the necessary imports

In [26]:
import numpy as np
import robotic as ry
import manipulation as manip

Now we load a predefined configuration with a robot arm and add a red box

In [27]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

midpoint = np.array([-0.105, 0.4, 0.705-.025+.1])
C.addFrame("box") \
    .setPosition(midpoint) \
    .setShape(ry.ST.ssBox, size=[0.08, 0.12, 0.08, 0.001]) \
    .setColor([1, 0, 0]) \
    .setContact(1) \
    .setMass(.1)

# For convenience, a few definitions:
gripper = "l_gripper"
box = "box"
table = "table"

C.view()

0

## Defining the push motion

Using the ManipullationModelling class we can firstly generate two key time slices, one represents the joint state at the start of the push motion in front of the box and another at the end target position of the box. From that we ca fill in the sub-motions, states in between the two key frames, with a few constraints. For the first sub-motion we want the robot to retract the push, this means moving away from teh object before we approach the object leaving us in the desired starting position for our push.

For the second sub-motion we initially ignore all collisions before defining our own explicit collision pairs, these being the robot's fingers with the table and the robot's hand with the table. We have to do this manually since the pushing motion requires us to collide against the box, otherwise we would not find any feasible motions.

In [28]:
def push(obj: str, target_pos: list[float], info: str="", vis: bool=False) -> tuple:
	M = manip.ManipulationModelling(C, f"push_motion_{info}", ['l_gripper'])
	M.setup_pick_and_place_waypoints(gripper, obj, 1e-1, accumulated_collisions=False)
	M.straight_push([1.,2.], obj, gripper, table)
	M.target_xy_position(2., obj, target_pos)
	M.solve()
	if not M.feasible:
		return False, []

	M1 = M.sub_motion(0, accumulated_collisions=False)
	M1.retractPush([.0, .15], gripper, .03)
	M1.approachPush([.85, 1.], gripper, .03)
	M1.keep_distance([.15,.85], obj, "l_finger1", .02)
	M1.keep_distance([.15,.85], obj, "l_finger2", .02)
	M1.keep_distance([.15,.85], obj, 'l_palm', .02)
	M1.keep_distance([], table, "l_finger1", .0)
	M1.keep_distance([], table, "l_finger2", .0)
	path1 = M1.solve()
	if not M1.feasible:
		return False, []

	M2 = M.sub_motion(1, accumulated_collisions=False)
	M2.komo.addObjective([], ry.FS.positionRel, [gripper, '_push_start'], ry.OT.eq, 1e1*np.array([[1,0,0],[0,0,1]]))
	path2 = M2.solve()
	if not M2.feasible:
		return False, []

	if vis:
		M1.play(C, 1.)
		C.attach(gripper, obj)
		M2.play(C, 1.)
		C.attach(table, obj)

	path = np.append(path1, path2)
	return True, path

## Visualizing and Executing the motion

We will want to test the output of our function with a real robot, or at least a physics simulation. For this porpuse we can define a bot object with which we will be able to interface with the robot.

In [29]:
bot = ry.BotOp(C, useRealRobot=False)
bot.home(C)

We will attempt to perform 30 push motions. For each successful motion we will first loop through all the calculated joint states in each path, attaching the box to the robot's gripper in the second sub-motion. ManipulationModelling has a convenient function for this called "play", which can be run once the optimizer has found a feasible path. After visualizing the motion in the configuration, we can test it in the simulation (or real robot).

In [30]:
attempt_count = 30
success_count = 0

for l in range(attempt_count):

	target_position = [
		midpoint[0] + np.random.uniform(-.15, .15),
		midpoint[1] + np.random.uniform(-.15, .15),
		0.]
	
	success, path = push(box, target_position, str(l), vis=True)

	if success:
		bot.move(path, [3])
		while bot.getTimeToEnd() > 0.:
			bot.sync(C)
		
	success_count += 1 if success else 0
	
print(f"Successful motions: {success_count}/{attempt_count}")

  -- feasible:push_motion_0
     { time: 0.016703, evals: 70, done: 1, feasible: 1, sos: 0.117092, f: 0, ineq: 0, eq: 4.39069e-06 }
  -- feasible:sub_motion_0--push_motion_0
     { time: 0.077371, evals: 46, done: 1, feasible: 1, sos: 1.0303, f: 0, ineq: 9.03723e-07, eq: 0.000180739 }
  -- feasible:sub_motion_1--push_motion_0
     { time: 0.008338, evals: 9, done: 1, feasible: 1, sos: 0.0845118, f: 0, ineq: 0, eq: 0.000131302 }
  -- infeasible:push_motion_1
     { time: 0.016034, evals: 52, done: 1, feasible: 0, sos: 0.243999, f: 0, ineq: 0.323061, eq: 0.770936 }
  -- infeasible:push_motion_2
     { time: 0.027716, evals: 102, done: 1, feasible: 0, sos: 0.149789, f: 0, ineq: 0.315816, eq: 0.803879 }
  -- feasible:push_motion_3
     { time: 0.010994, evals: 49, done: 1, feasible: 1, sos: 0.112156, f: 0, ineq: 0, eq: 5.2382e-05 }
  -- feasible:sub_motion_0--push_motion_3
     { time: 0.329212, evals: 200, done: 1, feasible: 1, sos: 2.27373, f: 0, ineq: 9.39003e-05, eq: 0.000622432 }
  --

KeyboardInterrupt: 